In [1]:
# Import libraries & setting path

import pandas as pd
from pathlib import Path

RAW = Path("../data/raw/online_retail_II.xlsx")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Load dataset

df = pd.read_excel(RAW)

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [9]:
# filter date only to 2010-2011 and country we are focusing only to UK

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df = df[(df["InvoiceDate"] >= "2010-01-01") & 
        (df["InvoiceDate"] <  "2012-01-01")]

df = df[df["Country"] == "United Kingdom"]

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom
45229,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0,United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom
45231,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45232,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN,United Kingdom


In [11]:
# --- Data Cleaning ---

# 1. Remove Duplicates
df = df.drop_duplicates()

# 2. Remove Cancelled & TEST Trx
df = df[~df["Invoice"].astype(str).str.startswith(("c", "C"), na=False)]
df = df[~df["StockCode"].astype(str).str.startswith(("TEST"), na=False)]

# 3. Remove rows with missing CustID
df = df.dropna(subset=["Customer ID"])

# 4. Remove Quantity ≤ 0 and Price ≤ 0
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]

# 5. Standardise product names & make new columns
df["Description"] = (
    df["Description"]
    .astype(str)
    .str.lower()
    .str.strip()
)

df["StockCode"] = df["StockCode"].astype(str).str.strip()
df["BasketID"] = df["Invoice"].astype(str)
df["Product"] = df["Description"].astype(str).str.strip().str.lower()
df["Revenue"] = df["Quantity"] * df["Price"]

# 6. Select only useful columns
cols = [
    "BasketID", "Customer ID", "InvoiceDate",
    "StockCode", "Product", "Quantity",
    "Price", "Revenue", "Country"
]
df = df[cols].copy()

In [13]:
print("Rows:", len(df))
print("Unique baskets:", df["BasketID"].nunique())
print("Unique customers:", df["Customer ID"].nunique())
print("Unique products:", df["Product"].nunique())

print("Any Quantity <= 0:", (df["Quantity"] <= 0).any())
print("Any Unit Price <= 0:", (df["Price"] <= 0).any())

df.head()

Rows: 336100
Unique baskets: 16178
Unique customers: 3861
Unique products: 4240
Any Quantity <= 0: False
Any Unit Price <= 0: False


,BasketID,Customer ID,InvoiceDate,StockCode,Product,Quantity,Price,Revenue,Country
45234,493414,14590.0,2010-01-04 10:28:00,21844,retro spot mug,36,2.55,91.8,United Kingdom
45235,493414,14590.0,2010-01-04 10:28:00,21533,retro spot large milk jug,12,4.25,51.0,United Kingdom
45236,493414,14590.0,2010-01-04 10:28:00,37508,new england ceramic cake server,2,2.55,5.1,United Kingdom
45237,493414,14590.0,2010-01-04 10:28:00,35001G,hand open shape gold,2,4.25,8.5,United Kingdom
45238,493414,14590.0,2010-01-04 10:28:00,21527,retro spot traditional teapot,12,6.95,83.4,United Kingdom


In [15]:
# Create derivative tables (for MBA purposes)

# 1. Basket Table (List of Products for MBA)

transactions = (
    df.groupby("BasketID")["Product"]
      .apply(lambda s: sorted(set(s.tolist())))
      .reset_index(name="products")
)

# 2. Basket-Product Quantity Table (for one-hot matrix)

basket_qty = (
    df.groupby(["BasketID", "Product"])["Quantity"]
      .sum()
      .reset_index()
)

# 3. Basket-level revenue summary

basket_revenue = (
    df.groupby("BasketID")["Revenue"]
      .sum()
      .reset_index()
      .rename(columns={"Revenue": "basket_revenue"})
)


In [17]:
# Save the processed data

transactions.to_parquet(OUT_DIR / "transactions.parquet", index=False)
basket_qty.to_parquet(OUT_DIR / "basket_qty.parquet", index=False)
basket_revenue.to_parquet(OUT_DIR / "basket_revenue.parquet", index=False)
df.to_parquet(OUT_DIR / "df_clean.parquet", index=False)